In [169]:
from collections import Counter, deque, defaultdict
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [84]:
def load_input(fn):
    with open(fn) as fh:
        h = fh.read().rstrip()
        
                    
    return h

def get_b_dict(fn):
    bd = {}
    with open(fn) as fh:
        for l in fh.readlines():
            x, y = l.rstrip().split(' = ')
            bd[x] = y
    return bd

In [203]:
def get_info(ind, b):
    version = int(b[ind:ind + 3], 2)
    ind += 3
    tp = int(b[ind:ind + 3], 2)
    ind += 3
    return version, tp, ind


def tp4(ind, b):
    #literal value, ind = first ind of the packet
    end = False
    packets = []
    while not end:
        p = b[ind: ind + 5]
        packets.append(p)
        ind += 5
        if p[0] == '0':
            end = True
    bval = ''
    for p in packets:
        bval += p[1:]
    val = int(bval, 2)
    return packets, val, ind


def tpO(ind, b):
    # ind = first ind of the packet
    if b[ind] == '0':
        ind += 1
        sp_len = int(b[ind:ind + 15], 2)
        ind += 15
        sps = b[ind: ind + sp_len]
#         ind += sp_len
        
        return 'len', sps, ind
    
    elif b[ind] == '1':
        ind += 1
        sp_n = int(b[ind: ind + 11], 2)
        ind += 11
        
        return 'count', sp_n, ind
    


def parse_packet(ind, b):
    v, tp, ind = get_info(ind, b)
#     print(v, tp, ind)
    if tp == 4:
        packets, val, ind = tp4(ind, b)
#         print('literal', val, ind)
        return v, val, None, None, ind, tp
    else:
        sp_type, val, ind = tpO(ind, b)
#         print(sp_type, val, ind)
        if sp_type == 'len':
            return v, None, val, None, ind, tp
        elif sp_type == 'count':
            return v, None, None, val, ind, tp
        
def op_ans(tn, vals):
    tn = int(tn)
    if tn == 0:
        return sum(vals)
    if tn == 1:
        return np.prod(vals)
    if tn == 2:
        return min(vals)
    if tn == 3:
        return max(vals)
    if tn == 5:
        if vals[0] > vals[1]:
            return 1
        else:
            return 0
    if tn == 6:
        if vals[0] < vals[1]:
            return 1
        else:
            return 0
    if tn == 7:
        if vals[0] == vals[1]:
            return 1
        else:
            return 0

In [134]:
b = '11101110000000001101010000001100100000100011000001100000'

parse_packet(0, b)

7 3 6
count 3 18


(7, None, None, 3, 18)

In [266]:
day = '16'
fn = f'../{day}/{day}.txt'
bd = get_b_dict('../16/b_dict.txt')

h = load_input(fn)


# h = '8A004A801A8002F478'
# # h = '620080001611562C8802118E34'
# # h = 'C0015000016115A2E0802F182340'
# h = 'A0016C880162017C3686B18A3D4780'

# h = '38006F45291200'
# h = 'EE00D40C823060'

h = '9C0141080250320F1802104A08'

b = ''
for hx in h:
    b += bd[hx]



# Part 2
'''
store dict {start_ind: value(?)}
[(typeID, value)]
just need like..tabs, so just record the level val somewhere
'''

ind = 0
v_sum = 0
level = 0
out_str = ''
lnums = defaultdict(int)
linds = defaultdict(int)
lop = defaultdict(int)
lvals = defaultdict(list)
ltype = defaultdict(str)
end = False
while not end:
    try:
        vn, litval, plen, pnum, ind, tp = parse_packet(ind, b)
    except:
        end = True
        break
    
    level_tabs = ''.join(['\t' for x in range(level)])
    out_str += f'{level_tabs}{tp}, {litval}\n'
    
    print(lop)
    print(lvals)
    print(ltype)
    print(lnums)
    print()
    
    if litval:
        lvals[level].append(litval)
        
    if ltype[level] == 'num':
        if lnums[level] > 0:
            lnums[level] = lnums[level] - 1
        elif lnums[level] == 0:
                val = op_ans(lop[level], lvals[level])
                lvals[level] = []
                level -= 1
                lvals[level].append(val)
                # when level goes back down, complete the operation
                
    elif ltype[level] == 'len':
        if ind >= linds[level]:
            val = op_ans(lop[level], lvals[level])
            lvals[level] = []
            level -= 1
            lvals[level].append(val)

    if pnum:
        level += 1
        lop[level] = tp
        lnums[level] = pnum
        ltype[level] = 'num'
        
    elif plen:
        level += 1
        lop[level] = tp
        linds[level] = ind + int(plen)
        ltype[level] = 'len'
    
    v_sum += vn
print(out_str)
print(op_ans(lop[1], lvals[1]))


defaultdict(<class 'int'>, {})
defaultdict(<class 'list'>, {})
defaultdict(<class 'str'>, {})
defaultdict(<class 'int'>, {})

defaultdict(<class 'int'>, {1: 7})
defaultdict(<class 'list'>, {})
defaultdict(<class 'str'>, {0: '', 1: 'len'})
defaultdict(<class 'int'>, {})

defaultdict(<class 'int'>, {1: 7, 2: 0})
defaultdict(<class 'list'>, {})
defaultdict(<class 'str'>, {0: '', 1: 'len', 2: 'num'})
defaultdict(<class 'int'>, {2: 2})

defaultdict(<class 'int'>, {1: 7, 2: 0})
defaultdict(<class 'list'>, {2: [1]})
defaultdict(<class 'str'>, {0: '', 1: 'len', 2: 'num'})
defaultdict(<class 'int'>, {2: 1})

defaultdict(<class 'int'>, {1: 7, 2: 0})
defaultdict(<class 'list'>, {2: [1, 3]})
defaultdict(<class 'str'>, {0: '', 1: 'len', 2: 'num'})
defaultdict(<class 'int'>, {2: 0})

defaultdict(<class 'int'>, {1: 7, 2: 1})
defaultdict(<class 'list'>, {2: [], 1: [4]})
defaultdict(<class 'str'>, {0: '', 1: 'len', 2: 'num'})
defaultdict(<class 'int'>, {2: 2})

defaultdict(<class 'int'>, {1: 7, 2: 1})


IndexError: list index out of range

In [247]:
print(op_ans(lop[1], lvals[1]))

IndexError: list index out of range

In [205]:
with open('../16/out_str.txt', 'w') as fh:
    fh.write(out_str)

In [160]:
# Part 1
ind = 0
v_sum = 0
for _ in range(4):

    vn, litval, plen, pnum, ind, tp = parse_packet(ind, b)
    v_sum += vn
    print(f'answer: {v_sum}')

answer: 6
answer: 12
answer: 14


ValueError: invalid literal for int() with base 2: ''

In [202]:
np.prod([1, 3, 4])

12

In [174]:
print(out_str)

0, None
	4, 1
	4, 2



In [ ]:
'''
3 bits - version
3 bits - type ID
    4: literal value
    other - operator
operator
    next bit 0 - next 15 bits
    next bit 1 - next 11 bits
    then sub packets
'''